In [1]:
# https://github.com/johanna23cct/integrated-CA2-MSc-2023094.git

In [2]:
#git remote add origin https://github.com/johanna23cct/integrated-CA2-MSc-2023094.git
#git branch -M main
#git push -u origin main

In [3]:
pip install Matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install bokeh

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install skforecast --user

Note: you may need to restart the kernel to use updated packages.


In [6]:
# Data manipulation
# ==============================================================================
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import download
from textblob import TextBlob

# Plots
# ==============================================================================
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

# Modeling and Forecasting
# ==============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
#from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

from joblib import dump, load

# Spark
# import SparkSession library 
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lower, concat_ws, regexp_replace, split, size, length, broadcast
from pyspark.sql.types import DoubleType, StructType, StructField, ArrayType, StringType, IntegerType, TimestampType
from pyspark.sql.types import *
import pyspark.sql.types as typ
import pyspark.sql.functions as fn
import pyspark.sql.functions as F


# Warnings configuration
# ==============================================================================
import warnings
# warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /home/hduser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
# SparkContext HOW I CAN SHOW THAT ... SC  It is not working to me, in the first time!
# that ok now, I know, is sc.master.  I was put only sc
sc.master

'local[*]'

In [8]:

spark = SparkSession.builder.appName("data_projectTweets").getOrCreate()

In [9]:
# creating a customer dataframe by declaring the schema and passing values


In [10]:
schema = StructType().add("Ids","integer").add("Date","string").add("Flag", "string").add("User",'string').add("Text", "string")

In [11]:
# ****************************************************************************************************************
# *                                  Start with the Data                                                         *
# ****************************************************************************************************************

In [20]:
# Big headache and colapse, to read the dataset
# I was load the file from my hadoop (local)
# df = spark.read.csv('home/hduser/Documnets/CA2', header+True, inferSchema=True)

In [21]:
#folowiong you can see the two way to load the dataset, 
#    (command)+(file://)+(/path/)+(filename)
#I will use direct from the path, to have one of five V' (Velocity) 

path = "/user1/ProjectTweets.csv"
#df = spark.read.csv("file:///home/hduser/Documents/CA2/ProjectTweets.csv", header=True, inferSchema = True)

In [22]:
#reading the data. i can't belive, it is working

In [23]:
tweets = sc.textFile("file:///home/hduser/Documents/CA2/ProjectTweets.csv")
header = tweets.first()

tweets = tweets \
    .filter(lambda row: row != header) \
    .map(lambda row: [int(elem) for elem in row.split(',')])

In [24]:
# creating the schema for my DataFrame
# was one  error because i didn' import pyspark.sql.types as typ
# but next did it, the command worked, great!

In [25]:
fields = [
    *[
        typ.StructField(h[1:-1], typ.IntegerType(), True)
        for h in header.split(',')
    ]
]
schema = typ.StructType(fields)

In [26]:
column_names  = ['_c0','Ids', 'Date', 'Flag', 'User', 'Text']    
full_df = pd.read_csv('file:///home/hduser/Documents/CA2/ProjectTweets.csv', header=None, names=column_names)

In [27]:
full_df.head()

,_c0,Ids,Date,Flag,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [28]:
# display the total number of rows data
total_rows = full_df.shape[0]
formatted_total_rows = "{:,}".format(total_rows)
print("Total Rows:", formatted_total_rows)

Total Rows: 1,600,000


In [29]:
# rename the file whit  row's title

In [30]:
full_df.to_csv("file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv")

In [31]:
Header_df = spark.read.csv("file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv", header=True, inferSchema = True)

In [32]:
# Drop unnecessary column  *****************************help********************************

In [33]:
#df = full_df.drop(['Flag', 'Unnamed: 0', ], axis=NO_QUERY)
#df.dropna(inplace=True)

Header_df.drop('Flag').show()

+---+----------+--------------------+---------------+--------------------+
|_c0|       Ids|                Date|           User|                Text|
+---+----------+--------------------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|         Karoli|@nationwideclass ...|
|  5|1467811372|Mon Apr 06 22:20:...|       joy_wolf|@Kwesidei not the...|
|  6|1467811592|Mon Apr 06 22:20:...|        mybirch|         Need a hug |
|  7|1467811594|Mon Apr 06 22:20:...|           coZZ|@LOLTrish hey  lo...|
|  8|1467811795|Mon Apr 06 22:20:...|2Hood4Hollywood|@Tatiana_K nope t...|
|  9|1467812025|Mon Apr 06 22:20:...|        mimismo|@twittera que me ...|
| 10|1467812416|Mon Apr 0

2023-11-05 00:47:32,452 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Ids, Date, User, Text
 Schema: _c0, Ids, Date, User, Text
Expected: _c0 but found: 
CSV file: file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv


In [34]:
schema = StructType().add("_c0","integer").add("Ids","integer").add("Date","string").add("Flag", "string").add("User",'string').add("Text", "string")

In [35]:
full_df.to_csv("file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv")

In [36]:
# Creating a UDF to apply VADER sentiment analysis to a Text column

In [37]:
def analyze_sentiment(Text):
    sid = SentimentIntensityAnalyzer()
    sentiment = sid.polarity_scores(Text)
    return sentiment['compound']

In [38]:
#register the UDF

In [39]:
sentiment_udf = udf(analyze_sentiment, DoubleType())

In [40]:
#Apply sentiment

In [41]:
Header_df = Header_df.withColumn("sentiment", sentiment_udf(Header_df["Text"]))

In [42]:
print(Header_df.show())

2023-11-05 00:47:49,297 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Ids, Date, Flag, User, Text
 Schema: _c0, Ids, Date, Flag, User, Text
Expected: _c0 but found: 
CSV file: file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv


+---+----------+--------------------+--------+---------------+--------------------+---------+
|_c0|       Ids|                Date|    Flag|           User|                Text|sentiment|
+---+----------+--------------------+--------+---------------+--------------------+---------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|  -0.0173|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|    -0.75|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|   0.4939|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|    -0.25|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|  -0.6597|
|  5|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|      0.0|
|  6|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |   0.4767|
|  7|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZ

2023-11-05 00:48:51,714 WARN python.PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 5 (TID 6): Attempting to kill Python Worker


In [43]:
#why i have Flag  again??
print(column_names[:5])

['_c0', 'Ids', 'Date', 'Flag', 'User']


In [82]:
#df = full_df.drop(['Flag', 'Unnamed: 0', ], axis=NO_QUERY)
#df.dropna(inplace=True)

Header_df.drop('Flag').show()

2023-11-05 00:51:18,093 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Ids, Date, User, Text
 Schema: _c0, Ids, Date, User, Text
Expected: _c0 but found: 
CSV file: file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv


+---+----------+--------------------+---------------+--------------------+--------------------+
|_c0|       Ids|                Date|           User|                Text|    tokenized_Tweets|
+---+----------+--------------------+---------------+--------------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|_TheSpecialOne_|@switchfoot http:...|[switchfoot, http...|
|  1|1467810672|Mon Apr 06 22:19:...|  scotthamilton|is upset that he ...|[is, upset, that,...|
|  2|1467810917|Mon Apr 06 22:19:...|       mattycus|@Kenichan I dived...|[Kenichan, I, div...|
|  3|1467811184|Mon Apr 06 22:19:...|        ElleCTF|my whole body fee...|[my, whole, body,...|
|  4|1467811193|Mon Apr 06 22:19:...|         Karoli|@nationwideclass ...|[nationwideclass,...|
|  5|1467811372|Mon Apr 06 22:20:...|       joy_wolf|@Kwesidei not the...|[Kwesidei, not, t...|
|  6|1467811592|Mon Apr 06 22:20:...|        mybirch|         Need a hug |      [Need, a, hug]|
|  7|1467811594|Mon Apr 06 22:20:...|   

In [88]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%>

In [89]:
# Creating my Dataframe (i did it in line 27 )

In [90]:
#        Basic Feature Extraction

In [91]:
Header_df = spark.read.csv("file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv", header=True, inferSchema = True)

In [92]:
Tweets = Header_df

In [93]:
Tweets = Tweets.withColumn("word_count", size(split(Tweets["Text"], " ")))

In [94]:
# Number of Words  

In [95]:
#Tutorial did't work because is a Panda code: TypeError: 'Column' object is not callable

#Tweets['word_count'] = Tweets['Text'].apply(lambda x: len(str(x).split("")))
#Tweets[['Text','word_count']].head()

In [96]:
Tweets.select("Text", "word_count").show()

+--------------------+----------+
|                Text|word_count|
+--------------------+----------+
|@switchfoot http:...|        20|
|is upset that he ...|        22|
|@Kenichan I dived...|        19|
|my whole body fee...|        11|
|@nationwideclass ...|        22|
|@Kwesidei not the...|         6|
|         Need a hug |         4|
|@LOLTrish hey  lo...|        24|
|@Tatiana_K nope t...|         7|
|@twittera que me ...|         6|
|spring break in p...|         8|
|I just re-pierced...|         6|
|@caregiving I cou...|        21|
|@octolinz16 It it...|        16|
|@smarrison i woul...|        23|
|@iamjazzyfizzle I...|        20|
|Hollis' death sce...|        19|
|about to file taxes |         5|
|@LettyA ahh ive a...|        12|
|@FakerPattyPattz ...|        13|
+--------------------+----------+
only showing top 20 rows



In [97]:
# Number of Characters

In [98]:
#Tutorial did't work because is a Panda code:TypeError: 'Column' object is not callable
#Tweets['char_count'] = Tweets['Text'].str.len()
## this also includes spaces
#Tweets[['Text','char_count']].head()

In [99]:
Tweets = Tweets.withColumn("char_count", length(Tweets["Text"]))

In [100]:
Tweets.select("Text", "char_count").show()

+--------------------+----------+
|                Text|char_count|
+--------------------+----------+
|@switchfoot http:...|       115|
|is upset that he ...|       111|
|@Kenichan I dived...|        89|
|my whole body fee...|        47|
|@nationwideclass ...|       111|
|@Kwesidei not the...|        29|
|         Need a hug |        11|
|@LOLTrish hey  lo...|        99|
|@Tatiana_K nope t...|        36|
|@twittera que me ...|        25|
|spring break in p...|        43|
|I just re-pierced...|        26|
|@caregiving I cou...|        94|
|@octolinz16 It it...|        77|
|@smarrison i woul...|       117|
|@iamjazzyfizzle I...|       103|
|Hollis' death sce...|        93|
|about to file taxes |        20|
|@LettyA ahh ive a...|        64|
|@FakerPattyPattz ...|        79|
+--------------------+----------+
only showing top 20 rows



In [101]:
#Basic Pre-processing

In [102]:
Tweets = Tweets.withColumn("Text", concat_ws("", lower(col("Text"))))

In [103]:
#Removing Punctuation 

In [104]:
Tweets = Tweets.withColumn("Text", regexp_replace(col("Text"), r'[^\w\s]', ''))

In [105]:
Tweets.select("Text").show()

+--------------------+
|                Text|
+--------------------+
|switchfoot httptw...|
|is upset that he ...|
|kenichan i dived ...|
|my whole body fee...|
|nationwideclass n...|
|kwesidei not the ...|
|         need a hug |
|loltrish hey  lon...|
|tatiana_k nope th...|
|twittera que me m...|
|spring break in p...|
|i just repierced ...|
|caregiving i coul...|
|octolinz16 it it ...|
|smarrison i would...|
|iamjazzyfizzle i ...|
|hollis death scen...|
|about to file taxes |
|lettya ahh ive al...|
|fakerpattypattz o...|
+--------------------+
only showing top 20 rows



In [106]:
# Removal of Stop Words
#Load stop word once and broadcast it to all worker nodes

In [107]:

stop  = stopwords.words('english')
broadcasted_stopwords = spark.sparkContext.broadcast(stop)

In [108]:
#Define a funcion to remove stopwords

In [109]:
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop]
    return " ".join(filtered_words)

In [110]:
#Register the UDF

In [111]:
remove_stopwords_udf = udf(remove_stopwords, StringType())

In [112]:
#Apply the UDF

In [113]:
Tweets = Tweets.withColumn("Text", remove_stopwords_udf(Tweets["Text"]))

In [114]:
Tweets.select("Text").show()

+--------------------+
|                Text|
+--------------------+
|switchfoot httptw...|
|upset cant update...|
|kenichan dived ma...|
|whole body feels ...|
|nationwideclass b...|
| kwesidei whole crew|
|            need hug|
|loltrish hey long...|
|tatiana_k nope didnt|
|  twittera que muera|
|spring break plai...|
|      repierced ears|
|caregiving couldn...|
|octolinz16 counts...|
|smarrison wouldve...|
|iamjazzyfizzle wi...|
|hollis death scen...|
|          file taxes|
|lettya ahh ive al...|
|fakerpattypattz o...|
+--------------------+
only showing top 20 rows



In [115]:
#*****************************************************************************************************************
#*                                       T O K E N I Z A T I O N                                                 *
#*****************************************************************************************************************

In [116]:
# UDF for Tokenization

In [117]:
def tokenize_Tweets(Text):
    blob = TextBlob(Text)
    return [str(word) for word in blob.words]

In [118]:
tokenize_udf = udf(tokenize_Tweets, ArrayType(StringType()))

In [119]:
#Apply the UDF

In [120]:
Header_df = Header_df.withColumn('tokenized_Tweets', tokenize_udf(Header_df['Text']))

In [81]:
Header_df.show()

2023-11-05 00:51:07,744 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Ids, Date, Flag, User, Text
 Schema: _c0, Ids, Date, Flag, User, Text
Expected: _c0 but found: 
CSV file: file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv


+---+----------+--------------------+--------+---------------+--------------------+--------------------+
|_c0|       Ids|                Date|    Flag|           User|                Text|    tokenized_Tweets|
+---+----------+--------------------+--------+---------------+--------------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|[switchfoot, http...|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|[is, upset, that,...|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|[Kenichan, I, div...|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|[my, whole, body,...|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|[nationwideclass,...|
|  5|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|[Kwesidei, not, t...|
|  6|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        m

In [122]:
#****************************************************************************************************************
#*                                        S T R E M M I N G                                                     *
#****************************************************************************************************************

In [124]:
st = PorterStemmer()

In [125]:
# Define a Udf for Stremming

In [127]:
def stem_text(text):
    stemmed_words = [st.stem(word) for word in text.split()]
    return " ".join(stemmed_words)

In [128]:
# Register the UDF

In [129]:
strem_udf = udf(stem_text, StringType())

In [130]:
#Create a Spark session

In [132]:
spark = SparkSession.builder.appName("StemmingExample").getOrCreate()


In [133]:
#DataFrame 

In [135]:
Tweets_stemmed = Tweets.withColumn("Stemmed_Text", stem_udf(Tweets["Text"]))

NameError: name 'stem_udf' is not defined

In [ ]:
# Show te resulting DataFrame

In [ ]:
Tweets_stemmed.show(truncate=False)

In [ ]:
#Data Procesising in panda

In [ ]:
#df = df.withColumn("Text", lower(df["text"]))
#df = df.withColumn("Text", regexp_replace(df["text"], "[^a-z0-9\\s]", ""))

In [ ]:
df = df.withColumn("Text", lower(df["text"]))
df = df.withColumn("Text", regexp_replace(df["text"], "[^a-z0-9\\s]", ""))

In [ ]:
rows_df.shape